# 消费水平预测

In [1]:
# coding=utf-8
"""
Credit by Liao Jialing

"""
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel

# 1. 读入清洗整理后的用户基本信息宽表

In [2]:
dataset = pd.read_csv('E:/user_all_cleared.csv')

D:\Users\Jialing Liao\AppData\Local\Continuum\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 将target放在第一列

In [3]:
shop_label = dataset.pop('shop_real_sum_avg')

In [4]:
dataset.insert(0,'shop_real_sum_avg',shop_label)

In [5]:
dataset.shape

(503862, 152)

In [6]:
dataset.head()

,shop_real_sum_avg,customerid,accountcash,accountbalance,integralbalance,consumetotal,consumetimes,lastconsume_diff_create_day,lastconsume_diff_create_hour,lastconsume_diff_now,...,age_from_song_language_3.0,age_from_song_language_not_known,shop_com_regioncode_310000.0,shop_com_regioncode_350000.0,shop_com_regioncode_440000.0,shop_com_regioncode_500000.0,shop_com_regioncode_not_known,shop_com_managetype_1.0,shop_com_managetype_2.0,shop_com_managetype_not_known
0,0,5792237,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,1
1,0,5792238,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,1
2,0,5792239,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,1
3,0,5792241,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,1
4,0,5792242,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,1


In [33]:
pd.value_counts(dataset['shop_real_sum_avg'])

0.000000      453894
39.000000       3389
12.000000       2741
43.000000       2564
10.000000       2097
8.000000        1605
15.000000       1536
18.000000       1485
20.000000       1473
16.000000       1149
6.000000        1028
30.000000        846
78.000000        590
11.000000        575
86.000000        560
28.000000        556
100.000000       552
58.000000        540
129.000000       502
13.000000        488
9.000000         451
25.000000        422
14.000000        411
68.000000        406
7.000000         405
500.000000       389
172.000000       382
117.000000       378
24.000000        371
38.000000        367
               ...  
349.333333         1
141.666667         1
44.538462          1
660.200000         1
195.857143         1
58.200000          1
167.500000         1
220.500000         1
147.250000         1
43.250000          1
162.689655         1
16.750000          1
64.600000          1
57.111111          1
924.764706         1
39.111111          1
147.666667   

# 2. 分出测试集和训练集

In [7]:
train = dataset[dataset['shop_real_sum_avg']!=0]
test = dataset[dataset['shop_real_sum_avg']==0]
test_id = test.customerid
test = test.drop('customerid',axis=1)
X = train.drop(['shop_real_sum_avg','customerid'],axis=1)
y = train.shop_real_sum_avg.values
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1026)

In [8]:
train.shape

(49968, 152)

In [9]:
test.shape

(453894, 151)

In [10]:
print(X_train.shape,X_test.shape,test.shape)

((39974, 150), (9994, 150), (453894, 151))


In [11]:
train.head()

,shop_real_sum_avg,customerid,accountcash,accountbalance,integralbalance,consumetotal,consumetimes,lastconsume_diff_create_day,lastconsume_diff_create_hour,lastconsume_diff_now,...,age_from_song_language_3.0,age_from_song_language_not_known,shop_com_regioncode_310000.0,shop_com_regioncode_350000.0,shop_com_regioncode_440000.0,shop_com_regioncode_500000.0,shop_com_regioncode_not_known,shop_com_managetype_1.0,shop_com_managetype_2.0,shop_com_managetype_not_known
301731,648.0,5792307,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,1,0,0
301732,11.5,5792313,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
301733,25.5,5792325,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
301737,10.0,5792340,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
301738,788.0,5792343,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0


# 3. 进行特征选择

## 由于消费金额是连续值，选RF regressor 而不是classifier

In [12]:
model = RandomForestRegressor(n_estimators=300)
model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=300, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

## 特征选择

In [13]:
important_ones = zip(X_train.columns.values, model.feature_importances_)
important_ones = sorted(important_ones, key=lambda x: x[1], reverse=True)

In [14]:
for r in important_ones:
    print r[0], r[1]

shop_original_sum_avg 0.836384756028
shop_real_sum_min 0.141422636353
shop_real_sum_max 0.0181771182479
shop_original_sum_max 0.00145969343726
shop_original_sum_min 0.00082844462926
shop_real_sum 0.000646188807819
shop_com_regioncode_350000.0 0.000234503067989
shop_original_sum 0.000168142446378
shop_com_roomnum 0.000139804564752
shop_companyid 9.68030790735e-05
shop_cnt 5.3449102812e-05
shop_weekday_sum 4.08383647175e-05
shop_com_cnt 2.97468307251e-05
shop_order_cnt 2.9741204472e-05
shop_night_sum 2.88797027324e-05
shop_before_dawn_sum 2.32186480316e-05
shop_weekend_sum 2.28641836328e-05
shop_weekday_cnt 1.70415148476e-05
shop_before_dawn_cnt 1.69983190028e-05
get_com_roomnum 1.62243300869e-05
shop_weekend_cnt 1.49818805172e-05
shop_night_cnt 1.4441928154e-05
use_coupon_order_sum 1.1348883742e-05
shop_afternoon_sum 1.10067509204e-05
shop_com_regioncode_440000.0 9.39478511809e-06
coupon_get_cnt 8.32839938918e-06
shop_com_regioncode_310000.0 6.54808591864e-06
get_coupon_type_cnt 6.20906

## 使用1e-5选出特征

In [20]:
THRESHOLD = 1e-5
important_features = pd.DataFrame(important_ones, columns=['feature', 'score'])
filtered_features = important_features.loc[important_features['score']>THRESHOLD, 'feature'].values

In [21]:
filtered_features

array(['shop_original_sum_avg', 'shop_real_sum_min', 'shop_real_sum_max',
       'shop_original_sum_max', 'shop_original_sum_min', 'shop_real_sum',
       'shop_com_regioncode_350000.0', 'shop_com_roomnum',
       'shop_original_sum', 'shop_companyid', 'shop_cnt',
       'shop_weekend_sum', 'shop_com_cnt', 'shop_weekday_sum',
       'shop_order_cnt', 'shop_night_sum', 'shop_before_dawn_sum',
       'get_com_roomnum', 'shop_before_dawn_cnt', 'shop_weekday_cnt',
       'shop_night_cnt', 'shop_weekend_cnt', 'shop_com_regioncode_440000.0'], dtype=object)

In [22]:
len(filtered_features)

23

## 使用1e-6选出特征

In [15]:
THRESHOLD = 1e-6
important_features = pd.DataFrame(important_ones, columns=['feature', 'score'])
filtered_features = important_features.loc[important_features['score']>THRESHOLD, 'feature'].values

In [16]:
filtered_features

array(['shop_original_sum_avg', 'shop_real_sum_min', 'shop_real_sum_max',
       'shop_original_sum_max', 'shop_original_sum_min', 'shop_real_sum',
       'shop_com_regioncode_350000.0', 'shop_original_sum',
       'shop_com_roomnum', 'shop_companyid', 'shop_cnt',
       'shop_weekday_sum', 'shop_com_cnt', 'shop_order_cnt',
       'shop_night_sum', 'shop_before_dawn_sum', 'shop_weekend_sum',
       'shop_weekday_cnt', 'shop_before_dawn_cnt', 'get_com_roomnum',
       'shop_weekend_cnt', 'shop_night_cnt', 'use_coupon_order_sum',
       'shop_afternoon_sum', 'shop_com_regioncode_440000.0',
       'coupon_get_cnt', 'shop_com_regioncode_310000.0',
       'get_coupon_type_cnt', 'use_company_id', 'source_company_cnt',
       'coupon_get_sum', 'use_coupon_type_cnt',
       'get_com_regioncode_350000.0',
       'wechat_province_\xe5\x9b\xbd\xe5\xa4\x96', 'source_company_id',
       'shop_afternoon_cnt', 'use_company_cnt', 'use_coupon_sum',
       'shop_com_regioncode_500000.0', 'shop_cross_com

In [17]:
len(filtered_features)

53

## 按选择到的特征分别处理训练集和测试集

In [18]:
X_test = X_test[filtered_features]

In [19]:
X_train = X_train[filtered_features]

In [20]:
X_train.head()

,shop_original_sum_avg,shop_real_sum_min,shop_real_sum_max,shop_original_sum_max,shop_original_sum_min,shop_real_sum,shop_com_regioncode_350000.0,shop_original_sum,shop_com_roomnum,shop_companyid,...,shop_wine_cnt,gender,wechat_country_未知,shop_com_managetype_2.0,wechat_province_福建,get_com_regioncode_not_known,get_com_managetype_not_known,get_com_regioncode_310000.0,get_com_regioncode_500000.0,wechat_province_未知
358065,12.000000,12,12,12,12,12,0,12,60,1101,...,0,2,1,0,0,0,0,0,1,1
315176,70.000000,70,70,70,70,70,0,70,80,438,...,0,2,1,0,0,0,0,1,0,1
354214,114.333333,12,316,316,12,343,0,343,38,6052,...,0,2,1,0,0,1,1,0,0,1
341939,39.000000,39,39,39,39,39,0,39,74,6491,...,0,2,1,0,0,0,0,0,0,1
314401,48.000000,48,48,48,48,48,0,48,60,7537,...,0,2,1,0,0,1,1,0,0,1


In [21]:
test = test[filtered_features]

# 3. Xgboost模型训练

In [126]:
params={
'booster':'gbtree',
'objective': 'reg:linear', # 这里用户消费金额是连续的，因此采用了默认的线性分类器
'gamma':0.05,  # 在树的叶子节点下一个分区的最小损失，越大算法模型越保守 [0:]
#'max_depth':12, # 构建树的深度 [1:]
#'lambda':450,  # L2 正则项权重
'subsample':0.4, # 采样训练数据，设置为0.5，随机选择一般的数据实例 (0:1]
'colsample_bytree':0.4, # 构建树树时的采样比率 (0:1]
#'min_child_weight':12, # 节点的最少特征数
'silent':1 ,
'eta': 0.3, # 如同学习率
'seed':710,
'nthread':4,# cpu 线程数,根据自己U的个数适当调整
}

In [127]:
plst = list(params.items())

In [128]:
num_rounds = 1000 # 迭代次数
xgtest = xgb.DMatrix(test)
xgtrain = xgb.DMatrix(X_train, label=y_train)
xgval = xgb.DMatrix(X_test, label=y_test)

### return训练和验证的标准误差

In [129]:
watchlist = [(xgtrain, 'train'),(xgval, 'val')]

### 训练模型

In [131]:
model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=50)

Will train until val error hasn't decreased in 50 rounds.
[0]	train-rmse:152.943481	val-rmse:148.323029
[1]	train-rmse:110.051735	val-rmse:106.258072
[2]	train-rmse:79.427391	val-rmse:76.671753
[3]	train-rmse:57.179379	val-rmse:55.117817
[4]	train-rmse:42.278782	val-rmse:40.717525
[5]	train-rmse:31.467247	val-rmse:30.612823
[6]	train-rmse:24.327295	val-rmse:23.850273
[7]	train-rmse:19.086205	val-rmse:19.135265
[8]	train-rmse:15.837515	val-rmse:16.315872
[9]	train-rmse:13.338251	val-rmse:14.521577
[10]	train-rmse:12.120469	val-rmse:13.568898
[11]	train-rmse:10.461782	val-rmse:12.621734
[12]	train-rmse:9.497982	val-rmse:11.966133
[13]	train-rmse:8.940937	val-rmse:11.784933
[14]	train-rmse:8.653774	val-rmse:11.634995
[15]	train-rmse:8.360218	val-rmse:11.431383
[16]	train-rmse:8.092566	val-rmse:11.435374
[17]	train-rmse:7.914980	val-rmse:11.269809
[18]	train-rmse:7.778362	val-rmse:11.065954
[19]	train-rmse:7.503653	val-rmse:10.894379
[20]	train-rmse:7.345752	val-rmse:10.817840
[21]	train-r

In [132]:
model.save_model('E:/consume.model')

In [133]:
preds = model.predict(xgtest,ntree_limit=model.best_iteration)

In [134]:
preds

array([ 2.47536612,  2.47536612,  2.47536612, ..., -2.75266743,
        0.80328512,  3.86955667], dtype=float32)

# 4. 对预测结果进行处理和保存

In [135]:
df = pd.DataFrame(preds)

In [136]:
df.columns = ['shop_real_sum_avg']

In [137]:
pd.value_counts(df['shop_real_sum_avg'])

 2.475366     401062
 2.967920      32801
 3.781528       6804
 2.825732       5495
 2.383048       2240
 2.875602       1939
-1.426011        393
 2.333179        326
-1.918565        287
-1.365257        223
-1.568198         68
-0.612404         49
 2.523153         39
-1.507444         38
-0.837469         34
-1.857811         34
-1.207404         29
 3.015707         27
 3.028674         24
 2.693973         24
-1.870778         24
-1.378224         23
-1.699958         19
-1.146649         17
-1.317470         17
 5.476693         15
 6.763845         14
-0.273955         14
 3.513440         12
-0.880491         12
               ...  
 3.005400          1
 3.494659          1
 5.143495          1
 3.978059          1
 0.455899          1
 6.485300          1
 3.535682          1
 1.742969          1
 2.394111          1
-4.028365          1
 13.571866         1
 2.030208          1
 2.585457          1
 8.927396          1
-0.455970          1
 0.874422          1
 2.547373    

In [138]:
consume = df.sort_values('shop_real_sum_avg',ascending=False)

In [139]:
consume.head()

,shop_real_sum_avg
285470,18.610174
292853,16.893156
288576,13.980245
287997,13.665714
285274,13.571866


In [140]:
df_y = pd.DataFrame(test_id)

In [142]:
df_y.shape

(453894, 1)

In [143]:
id_index = range(0,453894)

In [145]:
df_y = df_y.reindex(index=id_index)

In [146]:
consume_predicted = pd.concat([df_y,df],axis=1)

## 最终预测了453894项

In [147]:
consume_predicted.shape

(453894, 2)

In [148]:
consume_predicted.head()

,customerid,shop_real_sum_avg
0,5792237,2.475366
1,5792238,2.475366
2,5792239,2.475366
3,5792241,2.475366
4,5792242,2.475366


In [149]:
len(pd.value_counts(consume_predicted['shop_real_sum_avg']))

1342

In [151]:
consume = consume_predicted.sort_values('shop_real_sum_avg',ascending=False)
consume.head()

,customerid,shop_real_sum_avg
285470,5450143,18.610174
292853,5761915,16.893156
288576,4658155,13.980245
287997,5647514,13.665714
285274,4751000,13.571866


## 保存预测结果到consume_predicted.csv

In [150]:
consume_predicted.to_csv('E:/consume_predicted.csv')